# NAB unfolded
In this notebook we will explore the code fot Anomaly Detection, presented in [NAB](https://github.com/numenta/NAB), since is it seems to be the most reliable version.

[Here](https://www.youtube.com/watch?v=-N-2Cv-Q8h8) a presentation of the code `NAB/nab/detectors/numenta/numenta_detector.py`.

**PROCESS**: `input > create_model > encoder > SP > TM > AnomalyScore > AnomalyLH`

1. Everything is executed by **run.py**, which takes in a lot of information to start **runner.py**, which specifies the set of profiles, thresholds, and/or detectors --`numenta` in our case. 
1. To **LOAD the data .csv**, `runner.py` calls the script **corpus.py** which has two classes, *DataFile* followed by *Corpus*. *DataFile* loads the .csv which is called in the class *Corpus* in `#123`, `#135`. 
1. Afterwards, in **runner.py** `#93` calls the class *CorpusLaber* from **labeler.py**, which takes care af aggregating the groundtruth labels and saving to .json the timestamps for the anomaly.
1. **runner.py** executes the different detectors: *detect()* `#99` --> **numenta_detector.py** 
1. **numenta_detector.py** is more a wrapper that contains the childclass *NumentaDetector(AnomalyDetector)* `#42` which inherits methods from the parent class `AnomalyDetector` descripted in **base.py**, such `getAdditionalHeaders()` and `handleRecord()`
1. In **base.py**, *AnomalyDetector* takes as argument `dataSet` that is loaded using *Corpus* and among all, transforms our `pd.Dataframe`  into a `dict` *InputData*.
1. *handleRecord* in **anomaly-detector.py** calculates *Anomaly Score*, and *Anomaly LH*, called `rawScore` and `anomalyScore` respectively from `dict` *InputData* **????? #83**.
1. In **anomaly-detector.py**, the Model is initialized in `#113`, but it is not clear (to me), how is it feed with info. 
    ```
      def initialize(self):
        # Get config params, setting the RDSE resolution
        rangePadding = abs(self.inputMax - self.inputMin) * 0.2
        modelParams = getScalarMetricWithTimeOfDayAnomalyParams(
          metricData=[0],
          minVal=self.inputMin-rangePadding,
          maxVal=self.inputMax+rangePadding,
          minResolution=0.001,
          tmImplementation = "cpp"
        )["modelConfig"]
    ```
1. In **numenta_detector.py** `#116` Our first intuition is to get an overview of the fnc `from nupic.frameworks.opf.common_models.cluster_params import(getScalarMetricWithTimeOfDayAnomalyParams)`, 
[PATH](~/.local/share/virtualenvs/nupic-JMYHf7mp/lib/python2.7/site-packages/nupic/frameworks/opf/common_models/cluster_params.py`) since is the scripts that feeds the *Model*. 
1. **cluster_params.py**:
    - `#120`: **def _rangeGen(data, std=1)** calculates min/max values to use given the data and returns `minval, maxval` that are passed to `getScalarMetricWithTimeOfDayAnomalyParams` in `#29`
    - `#29`: **def getScalarMetricWithTimeOfDayAnomalyParams** returns a dict that will be the input of `nupic.frameworks.opf.model_factory.ModelFactory.create`.
    The dict contains `modelConfig` and `inferenceArgs` top-level properties. The value of the `modelConfig` property is for passing to `~nupic.frameworks.opf.model_factory.ModelFactory.create` method as the `modelConfig` parameter. The `inferenceArgs` property is for passing to the resulting model's `~nupic.frameworks.opf.model.Model.enableInference` method as the `inferenceArgs` parameter.
    
/ 1. In `numenta_detector.py #141` is defined the fnc `_setupEncoderParams(self, encoderParams)`, but some puzzle is missing.
    

### Load Data



**class Corpus** in courpus.py

In [1]:
import copy
import os
import pandas

from nab.util import (absoluteFilePaths,
                      createPath)

In [2]:
class Corpus(object):
  """
  Class for storing and manipulating a corpus of data where each datafile is
  stored as a DataFile object.
  """

  def __init__(self, srcRoot):
    """
    @param srcRoot    (string)    Source directory of corpus.
    """
    self.srcRoot = srcRoot
    self.dataFiles = self.getDataFiles()
    self.numDataFiles = len(self.dataFiles)


  def getDataFiles(self):
    """
    Collect all CSV data files from self.srcRoot directory.

    @return (dict)    Keys are relative paths (from self.srcRoot) and values are
                      the corresponding data files.
    """
    filePaths = absoluteFilePaths(self.srcRoot)
    dataSets = [DataFile(path) for path in filePaths if ".csv" in path]

    def getRelativePath(srcRoot, srcPath):
      return srcPath[srcPath.index(srcRoot)+len(srcRoot):]\
        .strip(os.path.sep).replace(os.path.sep, "/")

    return {getRelativePath(self.srcRoot, d.srcPath) : d for d in dataSets}


  def addColumn(self, columnName, data, write=False):
    """
    Add column to entire corpus given columnName and dictionary of data for each
    file in the corpus. If newRoot is given then corpus is copied and then
    modified.

    @param columnName   (string)  Name of the column in the datafile to add.

    @param data         (dict)    Dictionary containing key value pairs of a
                                  relative path and its corresponding
                                  datafile (as a pandas.Series).

    @param write        (boolean) Flag to decide whether to write corpus
                                  modificiations or not.
    """

    for relativePath in self.dataFiles.keys():
      self.dataFiles[relativePath].modifyData(
        columnName, data[relativePath], write=write)


  def removeColumn(self, columnName, write=False):
    """
    Remove column from entire corpus given columnName. If newRoot if given then
    corpus is copied and then modified.

    @param columnName   (string)  Name of the column in the datafile to add.

    @param write        (boolean) Flag to decide whether to write corpus
                                  modificiations or not.
    """
    for relativePath in self.dataFiles.keys():
      self.dataFiles[relativePath].modifyData(columnName, write=write)

  def copy(self, newRoot=None):
    """Copy corpus to a newRoot which cannot already exist.

    @param newRoot      (string)      Location of new directory to copy corpus
                                      to.
    """
    if newRoot[-1] != os.path.sep:
      newRoot += os.path.sep
    if os.path.isdir(newRoot):
      print "directory already exists"
      return None
    else:
      createPath(newRoot)

    newCorpus = Corpus(newRoot)
    for relativePath in self.dataFiles.keys():
      newCorpus.addDataSet(relativePath, self.dataFiles[relativePath])
    return newCorpus


  def addDataSet(self, relativePath, dataSet):
    """Add datafile to corpus given its realtivePath within the corpus.

    @param relativePath     (string)      Path of the new datafile relative to
                                          the corpus directory.

    @param datafile          (datafile)     Data set to be added to corpus.
    """
    self.dataFiles[relativePath] = copy.deepcopy(dataSet)
    newPath = self.srcRoot + relativePath
    createPath(newPath)
    self.dataFiles[relativePath].srcPath = newPath
    self.dataFiles[relativePath].write()
    self.numDataFiles = len(self.dataFiles)


  def getDataSubset(self, query):
    """
    Get subset of the corpus given a query to match the datafile filename or
    relative path.

    @param query        (string)      Search query for obtainin the subset of
                                      the corpus.

    @return             (dict)        Dictionary containing key value pairs of a
                                      relative path and its corresponding
                                      datafile.
    """
    ans = {}
    for relativePath in self.dataFiles.keys():
      if query in relativePath:
        ans[relativePath] = self.dataFiles[relativePath]
    return ans

class **DataFile** in corpus.py

In [3]:
class DataFile(object):
  """
  Class for storing and manipulating a single datafile.
  Data is stored in pandas.DataFrame
  """

  def __init__(self, srcPath):
    """
    @param srcPath (string)   Filename of datafile to read.
    """
    self.srcPath = srcPath  # path to file inclusive .csv

    self.fileName = os.path.split(srcPath)[1]  # select .csv name

    self.data = pandas.io.parsers.read_csv(self.srcPath,
                                           header=0, parse_dates=[0])  # header True, col[0] as date


  def write(self, newPath=None):
    """Write datafile to self.srcPath or newPath if given.

    @param newPath (string)   Path to write datafile to. If path is not given,
                              write to source path
    """

    path = newPath if newPath else self.srcPath
    self.data.to_csv(path, index=False)


  def modifyData(self, columnName, data=None, write=False):
    """Add columnName to datafile if data is given otherwise remove
    columnName.

    @param columnName (string)          Name of the column in the datafile to
                                        either add or remove.

    @param data       (pandas.Series)   Column data to be added to datafile.
                                        Data length should be as long as the
                                        length of other columns.

    @param write      (boolean) Flag to choose whether to write modifications to
                                source path.
    """
    if isinstance(data, pandas.Series):
      self.data[columnName] = data
    else:
      if columnName in self.data:
        del self.data[columnName]

    if write:
      self.write()


  def getTimestampRange(self, t1, t2):
    """Given timestamp range, get all records that are within that range.

    @param t1   (int)   Starting timestamp.

    @param t2   (int)   Ending timestamp.

    @return     (list)  Timestamp and value for each time stamp within the
                        timestamp range.
    """
    tmp = self.data[self.data["timestamp"] >= t1]
    ans = tmp[tmp["timestamp"] <= t2]["timestamp"].tolist()
    return ans


  def __str__(self):
    ans = ""
    ans += "path:                %s\n" % self.srcPath
    ans += "file name:           %s\n"% self.fileName
    ans += "data size:         ", self.data.shape()
    ans += "sample line: %s\n" % ", ".join(self.data[0])
    return ans

example of `DataFile` output

In [4]:
temp = DataFile("data/artificialNoAnomaly/art_daily_no_noise.csv")
vars(temp)

{'data':                timestamp     value
 0    2014-04-01 00:00:00  20.00000
 1    2014-04-01 00:05:00  20.00000
 2    2014-04-01 00:10:00  20.00000
 3    2014-04-01 00:15:00  20.00000
 4    2014-04-01 00:20:00  20.00000
 5    2014-04-01 00:25:00  20.00000
 6    2014-04-01 00:30:00  20.00000
 7    2014-04-01 00:35:00  20.00000
 8    2014-04-01 00:40:00  20.00000
 9    2014-04-01 00:45:00  20.00000
 10   2014-04-01 00:50:00  20.00000
 11   2014-04-01 00:55:00  20.00000
 12   2014-04-01 01:00:00  20.00000
 13   2014-04-01 01:05:00  20.00000
 14   2014-04-01 01:10:00  20.00000
 15   2014-04-01 01:15:00  20.00000
 16   2014-04-01 01:20:00  20.00000
 17   2014-04-01 01:25:00  20.00000
 18   2014-04-01 01:30:00  20.00000
 19   2014-04-01 01:35:00  20.00000
 20   2014-04-01 01:40:00  20.00000
 21   2014-04-01 01:45:00  20.00000
 22   2014-04-01 01:50:00  20.00000
 23   2014-04-01 01:55:00  20.00000
 24   2014-04-01 02:00:00  20.00000
 25   2014-04-01 02:05:00  20.00000
 26   2014-04-01 02:

exploring method **getDataFiles()**, in class Corpus, which calls the class **DataFile**

In [5]:
filePaths = absoluteFilePaths("data/")
dataSets = [DataFile(path) for path in filePaths if ".csv" in path] #DataFile loaded by DataFile class
dataSets[1].data  # access items in dataSets which is a list of DataFile objects

,timestamp,value
0,2014-04-01 00:00:00,18.622185
1,2014-04-01 00:05:00,8.163417
2,2014-04-01 00:10:00,13.292383
3,2014-04-01 00:15:00,11.667046
4,2014-04-01 00:20:00,12.940358
5,2014-04-01 00:25:00,14.277310
6,2014-04-01 00:30:00,12.791481
7,2014-04-01 00:35:00,12.563315
8,2014-04-01 00:40:00,16.059958
9,2014-04-01 00:45:00,15.544639


In [6]:
#list(os.walk("data"))

In [7]:
len(dataSets)

58

class **AnomalyDetector** in base.py

In [8]:
import abc
import os
import pandas
import sys

from datetime import datetime
from nab.util import createPath, getProbationPeriod

class AnomalyDetector(object):
  """
  Base class for all anomaly detectors. When inheriting from this class please
  take note of which methods MUST be overridden, as documented below.
  """
  __metaclass__ = abc.ABCMeta

  def __init__( self,
                dataSet,
                probationaryPercent):

    self.dataSet = dataSet
    self.probationaryPeriod = getProbationPeriod(
      probationaryPercent, dataSet.data.shape[0])

    self.inputMin = self.dataSet.data["value"].min()
    self.inputMax = self.dataSet.data["value"].max()


  def initialize(self):
    """Do anything to initialize your detector in before calling run.

    Pooling across cores forces a pickling operation when moving objects from
    the main core to the pool and this may not always be possible. This function
    allows you to create objects within the pool itself to avoid this issue.
    """
    pass

  def getAdditionalHeaders(self):
    """
    Returns a list of strings. Subclasses can add in additional columns per
    record.

    This method MAY be overridden to provide the names for those
    columns.
    """
    return []


  @abc.abstractmethod
  def handleRecord(self, inputData):
    """
    Returns a list [anomalyScore, *]. It is required that the first
    element of the list is the anomalyScore. The other elements may
    be anything, but should correspond to the names returned by
    getAdditionalHeaders().

    This method MUST be overridden by subclasses
    """
    raise NotImplementedError


  def getHeader(self):
    """
    Gets the outputPath and all the headers needed to write the results files.
    """
    headers = ["timestamp",
                "value",
                "anomaly_score"]

    headers.extend(self.getAdditionalHeaders())

    return headers


  def run(self):
    """
    Main function that is called to collect anomaly scores for a given file.
    """

    headers = self.getHeader()

    rows = []
    for i, row in self.dataSet.data.iterrows():

      inputData = row.to_dict()

      detectorValues = self.handleRecord(inputData)

      outputRow = list(row) + list(detectorValues)

      rows.append(outputRow)

      # Progress report
      if (i % 1000) == 0:
        print ".",
        sys.stdout.flush()

    ans = pandas.DataFrame(rows, columns=headers)
    return ans

In [9]:
#AnomalyDetector(dataSets[0])

We can not simply call the class.

**run()** method in class AnomalyDetector which calls the class **Corpus** and outputs a dict

In [10]:
row = []

for i, row in dataSets[1].data.iterrows():
    inputData = row.to_dict()
    print i
    print row

0
timestamp    2014-04-01 00:00:00
value                    18.6222
Name: 0, dtype: object
1
timestamp    2014-04-01 00:05:00
value                    8.16342
Name: 1, dtype: object
2
timestamp    2014-04-01 00:10:00
value                    13.2924
Name: 2, dtype: object
3
timestamp    2014-04-01 00:15:00
value                     11.667
Name: 3, dtype: object
4
timestamp    2014-04-01 00:20:00
value                    12.9404
Name: 4, dtype: object
5
timestamp    2014-04-01 00:25:00
value                    14.2773
Name: 5, dtype: object
6
timestamp    2014-04-01 00:30:00
value                    12.7915
Name: 6, dtype: object
7
timestamp    2014-04-01 00:35:00
value                    12.5633
Name: 7, dtype: object
8
timestamp    2014-04-01 00:40:00
value                      16.06
Name: 8, dtype: object
9
timestamp    2014-04-01 00:45:00
value                    15.5446
Name: 9, dtype: object
10
timestamp    2014-04-01 00:50:00
value                     17.166
Name: 10, dtype: obje

108
timestamp    2014-04-01 09:00:00
value                    13.3503
Name: 108, dtype: object
109
timestamp    2014-04-01 09:05:00
value                     11.283
Name: 109, dtype: object
110
timestamp    2014-04-01 09:10:00
value                     15.019
Name: 110, dtype: object
111
timestamp    2014-04-01 09:15:00
value                    11.1697
Name: 111, dtype: object
112
timestamp    2014-04-01 09:20:00
value                    10.5961
Name: 112, dtype: object
113
timestamp    2014-04-01 09:25:00
value                    16.0357
Name: 113, dtype: object
114
timestamp    2014-04-01 09:30:00
value                    8.44358
Name: 114, dtype: object
115
timestamp    2014-04-01 09:35:00
value                    9.04245
Name: 115, dtype: object
116
timestamp    2014-04-01 09:40:00
value                    8.07467
Name: 116, dtype: object
117
timestamp    2014-04-01 09:45:00
value                    9.68238
Name: 117, dtype: object
118
timestamp    2014-04-01 09:50:00
value        

Name: 233, dtype: object
234
timestamp    2014-04-01 19:30:00
value                    8.31171
Name: 234, dtype: object
235
timestamp    2014-04-01 19:35:00
value                    17.1138
Name: 235, dtype: object
236
timestamp    2014-04-01 19:40:00
value                     16.671
Name: 236, dtype: object
237
timestamp    2014-04-01 19:45:00
value                    10.8494
Name: 237, dtype: object
238
timestamp    2014-04-01 19:50:00
value                    15.7954
Name: 238, dtype: object
239
timestamp    2014-04-01 19:55:00
value                    10.3634
Name: 239, dtype: object
240
timestamp    2014-04-01 20:00:00
value                    8.44363
Name: 240, dtype: object
241
timestamp    2014-04-01 20:05:00
value                    10.2125
Name: 241, dtype: object
242
timestamp    2014-04-01 20:10:00
value                    9.96492
Name: 242, dtype: object
243
timestamp    2014-04-01 20:15:00
value                    11.7152
Name: 243, dtype: object
244
timestamp    2014-04-

Name: 366, dtype: object
367
timestamp    2014-04-02 06:35:00
value                    14.4647
Name: 367, dtype: object
368
timestamp    2014-04-02 06:40:00
value                    8.70448
Name: 368, dtype: object
369
timestamp    2014-04-02 06:45:00
value                    17.4868
Name: 369, dtype: object
370
timestamp    2014-04-02 06:50:00
value                    18.5765
Name: 370, dtype: object
371
timestamp    2014-04-02 06:55:00
value                    10.7142
Name: 371, dtype: object
372
timestamp    2014-04-02 07:00:00
value                     12.951
Name: 372, dtype: object
373
timestamp    2014-04-02 07:05:00
value                    8.24348
Name: 373, dtype: object
374
timestamp    2014-04-02 07:10:00
value                    15.0478
Name: 374, dtype: object
375
timestamp    2014-04-02 07:15:00
value                    12.5532
Name: 375, dtype: object
376
timestamp    2014-04-02 07:20:00
value                    10.9706
Name: 376, dtype: object
377
timestamp    2014-04-

Name: 503, dtype: object
504
timestamp    2014-04-02 18:00:00
value                    12.9298
Name: 504, dtype: object
505
timestamp    2014-04-02 18:05:00
value                    11.0185
Name: 505, dtype: object
506
timestamp    2014-04-02 18:10:00
value                    12.3249
Name: 506, dtype: object
507
timestamp    2014-04-02 18:15:00
value                    12.2654
Name: 507, dtype: object
508
timestamp    2014-04-02 18:20:00
value                    11.1555
Name: 508, dtype: object
509
timestamp    2014-04-02 18:25:00
value                    10.9544
Name: 509, dtype: object
510
timestamp    2014-04-02 18:30:00
value                    17.4717
Name: 510, dtype: object
511
timestamp    2014-04-02 18:35:00
value                    8.76848
Name: 511, dtype: object
512
timestamp    2014-04-02 18:40:00
value                    15.4537
Name: 512, dtype: object
513
timestamp    2014-04-02 18:45:00
value                    9.90102
Name: 513, dtype: object
514
timestamp    2014-04-

...

Name: 3532, dtype: object
3533
timestamp    2014-04-13 06:25:00
value                    9.43808
Name: 3533, dtype: object
3534
timestamp    2014-04-13 06:30:00
value                    10.3658
Name: 3534, dtype: object
3535
timestamp    2014-04-13 06:35:00
value                    15.2185
Name: 3535, dtype: object
3536
timestamp    2014-04-13 06:40:00
value                    12.2355
Name: 3536, dtype: object
3537
timestamp    2014-04-13 06:45:00
value                    17.2012
Name: 3537, dtype: object
3538
timestamp    2014-04-13 06:50:00
value                    18.4495
Name: 3538, dtype: object
3539
timestamp    2014-04-13 06:55:00
value                     16.862
Name: 3539, dtype: object
3540
timestamp    2014-04-13 07:00:00
value                     12.153
Name: 3540, dtype: object
3541
timestamp    2014-04-13 07:05:00
value                    16.6756
Name: 3541, dtype: object
3542
timestamp    2014-04-13 07:10:00
value                    11.8659
Name: 3542, dtype: object
3543

timestamp    2014-04-13 16:25:00
value                    15.6495
Name: 3653, dtype: object
3654
timestamp    2014-04-13 16:30:00
value                    10.7802
Name: 3654, dtype: object
3655
timestamp    2014-04-13 16:35:00
value                    11.4255
Name: 3655, dtype: object
3656
timestamp    2014-04-13 16:40:00
value                    9.33189
Name: 3656, dtype: object
3657
timestamp    2014-04-13 16:45:00
value                    17.5113
Name: 3657, dtype: object
3658
timestamp    2014-04-13 16:50:00
value                    13.4279
Name: 3658, dtype: object
3659
timestamp    2014-04-13 16:55:00
value                    14.2936
Name: 3659, dtype: object
3660
timestamp    2014-04-13 17:00:00
value                    14.2638
Name: 3660, dtype: object
3661
timestamp    2014-04-13 17:05:00
value                    12.5154
Name: 3661, dtype: object
3662
timestamp    2014-04-13 17:10:00
value                    10.1178
Name: 3662, dtype: object
3663
timestamp    2014-04-13 17:15:

timestamp    2014-04-14 02:30:00
value                    9.02661
Name: 3774, dtype: object
3775
timestamp    2014-04-14 02:35:00
value                    9.89923
Name: 3775, dtype: object
3776
timestamp    2014-04-14 02:40:00
value                     8.1352
Name: 3776, dtype: object
3777
timestamp    2014-04-14 02:45:00
value                    14.5302
Name: 3777, dtype: object
3778
timestamp    2014-04-14 02:50:00
value                    17.7541
Name: 3778, dtype: object
3779
timestamp    2014-04-14 02:55:00
value                    15.3773
Name: 3779, dtype: object
3780
timestamp    2014-04-14 03:00:00
value                    18.8024
Name: 3780, dtype: object
3781
timestamp    2014-04-14 03:05:00
value                    8.69337
Name: 3781, dtype: object
3782
timestamp    2014-04-14 03:10:00
value                     17.933
Name: 3782, dtype: object
3783
timestamp    2014-04-14 03:15:00
value                    16.9548
Name: 3783, dtype: object
3784
timestamp    2014-04-14 03:20:

Name: 3890, dtype: object
3891
timestamp    2014-04-14 12:15:00
value                    17.3098
Name: 3891, dtype: object
3892
timestamp    2014-04-14 12:20:00
value                    15.4631
Name: 3892, dtype: object
3893
timestamp    2014-04-14 12:25:00
value                    18.4942
Name: 3893, dtype: object
3894
timestamp    2014-04-14 12:30:00
value                    13.6211
Name: 3894, dtype: object
3895
timestamp    2014-04-14 12:35:00
value                     18.052
Name: 3895, dtype: object
3896
timestamp    2014-04-14 12:40:00
value                    18.6459
Name: 3896, dtype: object
3897
timestamp    2014-04-14 12:45:00
value                    17.3187
Name: 3897, dtype: object
3898
timestamp    2014-04-14 12:50:00
value                    11.4073
Name: 3898, dtype: object
3899
timestamp    2014-04-14 12:55:00
value                    10.1584
Name: 3899, dtype: object
3900
timestamp    2014-04-14 13:00:00
value                    14.0293
Name: 3900, dtype: object
3901

Name: 4010, dtype: object
4011
timestamp    2014-04-14 22:15:00
value                    17.6286
Name: 4011, dtype: object
4012
timestamp    2014-04-14 22:20:00
value                    10.0829
Name: 4012, dtype: object
4013
timestamp    2014-04-14 22:25:00
value                    14.0266
Name: 4013, dtype: object
4014
timestamp    2014-04-14 22:30:00
value                    8.26358
Name: 4014, dtype: object
4015
timestamp    2014-04-14 22:35:00
value                    16.4064
Name: 4015, dtype: object
4016
timestamp    2014-04-14 22:40:00
value                    14.3901
Name: 4016, dtype: object
4017
timestamp    2014-04-14 22:45:00
value                    14.8815
Name: 4017, dtype: object
4018
timestamp    2014-04-14 22:50:00
value                    8.56893
Name: 4018, dtype: object
4019
timestamp    2014-04-14 22:55:00
value                    10.3214
Name: 4019, dtype: object
4020
timestamp    2014-04-14 23:00:00
value                    8.81764
Name: 4020, dtype: object
4021

In [11]:
inputData #dictionary

{'timestamp': Timestamp('2014-04-14 23:55:00'), 'value': 10.0516211375}

## Create Model 

from **cluster_params.py**

In [12]:
import numpy as np

def _rangeGen(data, std=1):
  """
  Return reasonable min/max values to use given the data.
  """
  dataStd = np.std(data)
  if dataStd == 0:
    dataStd = 1
  minval = np.min(data) -  std * dataStd
  maxval = np.max(data) +  std * dataStd
  return minval, maxval

In [13]:
_rangeGen(dataSets[0].data['value'])

(-7.9508824900404171, 107.95085177004042)

In [14]:
import pixiedust

Pixiedust database opened successfully


In [15]:
import json
import os
from pkg_resources import resource_stream

def getScalarMetricWithTimeOfDayAnomalyParams(metricData,
                                              minVal=None,
                                              maxVal=None,
                                              minResolution=None,
                                              tmImplementation = "cpp"):
  
  # Default values
  if minResolution is None:
    minResolution = 0.001

  # Compute min and/or max from the data if not specified
  if minVal is None or maxVal is None:
    compMinVal, compMaxVal = _rangeGen(metricData)
    if minVal is None:
      minVal = compMinVal
    if maxVal is None:
      maxVal = compMaxVal

  # Handle the corner case where the incoming min and max are the same
  if minVal == maxVal:
    maxVal = minVal + 1

  # Load model parameters and update encoder params
  if (tmImplementation is "cpp"):
    paramFileRelativePath = os.path.join(
      "anomaly_params_random_encoder",
      "best_single_metric_anomaly_params_cpp.json")
  elif (tmImplementation is "tm_cpp"):
    paramFileRelativePath = os.path.join(
      "anomaly_params_random_encoder",
      "best_single_metric_anomaly_params_tm_cpp.json")
  else:
    raise ValueError("Invalid string for tmImplementation. Try cpp or tm_cpp")

  with resource_stream(__name__, paramFileRelativePath) as infile:
    paramSet = json.load(infile)

  _fixupRandomEncoderParams(paramSet, minVal, maxVal, minResolution)

  return paramSet


In [16]:
import os
paramFileRelativePath = os.path.join("anomaly_params_random_encoder", "best_single_metric_anomaly_params_tm_cpp.json")
print paramFileRelativePath

anomaly_params_random_encoder/best_single_metric_anomaly_params_tm_cpp.json


In [17]:
#%%pixie_debugger
getScalarMetricWithTimeOfDayAnomalyParams(dataSets[0].data['value'], tmImplementation = "cpp")

IOError: [Errno 2] No such file or directory: 'anomaly_params_random_encoder/best_single_metric_anomaly_params_cpp.json'

NameError: name 'params' is not defined